In [ ]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None, 'max_rows', None, 'expand_frame_repr', False)

In [ ]:
def mod_columns(df):
    result = df[['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'Carrier', 'Origin', 'OriginCityName', 
                 'Dest', 'DestCityName', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepartureDelayGroups', 
                 'DepTimeBlk', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrivalDelayGroups', 'ArrTimeBlk', 
                 'Cancelled', 'Diverted','CRSElapsedTime','Distance', 'CarrierDelay', 'WeatherDelay', 
                 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']]
    result = result.rename(columns={'DayOfWeek': 'DayofWeek'})
    return result

In [ ]:
month_data = []
for i in range(1, 13):
    filename = 'On_Time_On_Time_Performance_2016_' + str(i) + '.csv'
    data = pd.read_csv(filename)
    month_data.append(mod_columns(data))

In [ ]:
data = pd.concat(month_data)

In [ ]:
data.head()

In [ ]:
def countyes(x):
    count = 0
    for i in x:
        if i >= 1.0:
            count += 1
    return count

In [ ]:
def get_origin_agg(data):
    agg = data.agg({'Year': 'count', 
                    'DepDelay': 'mean', 
                    'CarrierDelay': 'sum', 
                    'WeatherDelay': 'sum', 
                    'NASDelay': 'sum', 
                    'SecurityDelay': 'sum', 
                    'LateAircraftDelay': 'sum', 
                    'DepartureDelayGroups': lambda x : countyes(x), 
                    'Cancelled': lambda x : countyes(x), 
                    'Diverted': lambda x : countyes(x)
                   }).rename(columns={'Year': 'Count', 
                                      'DepartureDelayGroups': '15minDepDelay'})
    agg = agg[agg['Count'] > 1]
    total_delay = agg[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 
                       'LateAircraftDelay']].sum(axis=1)
    agg['CarrierDelay'] = agg['CarrierDelay'] / total_delay
    agg['WeatherDelay'] = agg['WeatherDelay'] / total_delay
    agg['NASDelay'] = agg['NASDelay'] / total_delay
    agg['SecurityDelay'] = agg['SecurityDelay'] / total_delay
    agg['LateAircraftDelay'] = agg['LateAircraftDelay'] / total_delay
    agg['15minDepDelay'] = agg['15minDepDelay'] / agg['Count']
    agg['Cancelled'] = agg['Cancelled'] / agg['Count']
    agg['Diverted'] = agg['Diverted'] / agg['Count']
    return agg

In [ ]:
def get_dest_agg(data):
    agg = data.agg({'Year': 'count', 
                    'ArrDelay': 'mean', 
                    'ArrivalDelayGroups': lambda x : countyes(x), 
                   }).rename(columns={'Year': 'Count2', 
                                      'ArrivalDelayGroups': '15minArrDelay'})
    agg = agg[agg['Count2'] > 1]
    agg['15minArrDelay'] = agg['15minArrDelay'] / agg['Count2']
    return agg

In [ ]:
origin = data.groupby('Origin', as_index=False)
dest = data.groupby('Dest', as_index=False)

In [ ]:
origin_agg = get_origin_agg(origin)
origin_agg

In [ ]:
dest_agg = get_dest_agg(dest)
dest_agg

In [ ]:
agg = pd.merge(origin_agg, dest_agg, left_on='Origin', right_on='Dest', how='inner')
# agg.to_csv('summary.csv', index=False)
agg

In [ ]:
aplist = agg['Origin']
aplist.to_csv("airport_with_data.csv", index=False)

In [ ]:
def get_origin_data(name):
    
    group = origin.get_group(name)
        
    year = origin_agg[origin_agg['Origin'] == name]
    year = year.rename(columns={'Origin': 'Time'})
    year['Time'] = '2016'
    year.insert(0, 'Type', 'y')
    
    month = group.groupby('Month', as_index=False)
    month_agg = get_origin_agg(month)
    month_agg = month_agg.rename(columns={'Month': 'Time'})
    month_agg.insert(0, 'Type', 'm')
    
    week = group.groupby('DayofWeek', as_index=False)
    week_agg = get_origin_agg(week)
    week_agg = week_agg.rename(columns={'DayofWeek': 'Time'})
    week_agg.insert(0, 'Type', 'd')
    
    hour = group.groupby('DepTimeBlk', as_index=False)
    hour_agg = get_origin_agg(hour)
    hour_agg = hour_agg.rename(columns={'DepTimeBlk': 'Time'})
    hour_agg.insert(0, 'Type', 'h')
    
    result = pd.concat([year, month_agg, week_agg, hour_agg])
    result = result.reset_index(drop=True)
    return result

In [ ]:
def get_dest_data(name):
    
    group = dest.get_group(name)
        
    year = dest_agg[dest_agg['Dest'] == name]
    year = year.rename(columns={'Dest': 'Time'})
    year['Time'] = '2016'
    year.insert(0, 'Type', 'y')
    
    month = group.groupby('Month', as_index=False)
    month_agg = get_dest_agg(month)
    month_agg = month_agg.rename(columns={'Month': 'Time'})
    month_agg.insert(0, 'Type', 'm')
    
    week = group.groupby('DayofWeek', as_index=False)
    week_agg = get_dest_agg(week)
    week_agg = week_agg.rename(columns={'DayofWeek': 'Time'})
    week_agg.insert(0, 'Type', 'd')
    
    hour = group.groupby('DepTimeBlk', as_index=False)
    hour_agg = get_dest_agg(hour)
    hour_agg = hour_agg.rename(columns={'DepTimeBlk': 'Time'})
    hour_agg.insert(0, 'Type', 'h')
    
    result = pd.concat([year, month_agg, week_agg, hour_agg])
    result = result.reset_index(drop=True)
    return result

In [ ]:
origin_data = get_origin_data('SLC')
origin_data

In [ ]:
dest_data = get_dest_data('SLC')
dest_data

In [ ]:
concat_data = pd.merge(origin_data, dest_data, on=['Type', 'Time'], how='outer')
concat_data = concat_data.fillna(0)
concat_data

In [ ]:
for name in list:
    filename = name + '.csv'
    origin_data = get_origin_data(name)
    dest_data = get_dest_data(name)
    concat_data = pd.merge(origin_data, dest_data, on=['Type', 'Time'], how='outer')
    concat_data = concat_data.fillna(0)
    concat_data.to_csv(filename, index=False)

In [ ]:
# def get_data(name):
    
#     year = agg[agg['Origin'] == name]
#     year = year.rename(columns={'Origin': 'Time'})
#     year['Time'] = '2016'
#     year.insert(0, 'Type', 'y')
    
#     origin_month = origin.get_group(name).groupby('Month', as_index=False)
#     origin_month_agg = get_origin_agg(origin_month)
#     dest_month = dest.get_group(name).groupby('Month', as_index=False)
#     dest_month_agg = get_dest_agg(dest_month)
#     month_agg = pd.concat([origin_month_agg, dest_month_agg[['ArrDelay', '15minArrDelay']]], axis=1, join='outer')
#     month_agg = month_agg.rename(columns={'Month': 'Time'})
#     month_agg.insert(0, 'Type', 'm')
    
#     origin_week = origin.get_group(name).groupby('DayofWeek', as_index=False)
#     origin_week_agg = get_origin_agg(origin_week)
#     dest_week = dest.get_group(name).groupby('DayofWeek', as_index=False)
#     dest_week_agg = get_dest_agg(dest_week)
#     week_agg = pd.concat([origin_week_agg, dest_week_agg[['ArrDelay', '15minArrDelay']]], axis=1, join='outer')
#     week_agg = week_agg.rename(columns={'DayofWeek': 'Time'})
#     week_agg.insert(0, 'Type', 'd')
    
#     origin_hour = origin.get_group(name).groupby('DepTimeBlk', as_index=False)
#     origin_hour_agg = get_origin_agg(origin_hour)
#     dest_hour = dest.get_group(name).groupby('DepTimeBlk', as_index=False)
#     dest_hour_agg = get_dest_agg(dest_hour)
#     hour_agg = pd.concat([origin_hour_agg, dest_hour_agg[['ArrDelay', '15minArrDelay']]], axis=1, join='outer')
#     hour_agg = hour_agg.rename(columns={'DepTimeBlk': 'Time'})
#     hour_agg.insert(0, 'Type', 'h')

#     result = pd.concat([year, month_agg, week_agg, hour_agg])
#     result = result.reset_index(drop=True)
#     return result